In [1]:
import os
from scipy.spatial.distance import cdist
from skimage.io import imread
from skimage.feature import hog
from sklearn.cluster import MiniBatchKMeans

In [2]:
class image_label():

    def __init__(self, img=None,number=None,label=None):
        self.img = img
        self.number=number
        self.label=label

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
categories = ['bedroom', 'Coast', 'Forest', 'Highway', 'industrial',
           'Insidecity', 'kitchen', 'livingroom', 'Mountain', 'Office',
           'OpenCountry', 'store', 'Street', 'Suburb', 'TallBuilding'] 

train_img_set=[]
test_img_set=[]

for j in range(15):
    for i in range(100):
        img_PIL = Image.open("E:\\Course\\Computer Vision\\Coursework3\\training\\training\\"+ categories[j] +"\\"+str(i)+".jpg")
        #img_PIL = np.array(img_PIL)
        img=image_label(img_PIL,i,j)
        train_img_set.append(img)
error=0

for i in range(2988):
    try:
        img_PIL = Image.open("E:\\Course\\Computer Vision\\Coursework3\\testing\\"+str(i)+".jpg")
        img=image_label(img_PIL,i)
        test_img_set.append(img)
        error=error+1
    except Exception as e:
        pass
    continue
   

In [4]:
from sklearn.cluster import MiniBatchKMeans
from tqdm import tqdm
import random


def patches_feature(image_set):
    
    rng = np.random.RandomState(0)
    kmeans = MiniBatchKMeans(n_clusters = 200, random_state = rng)
    
    for i in tqdm(range(len(image_set))):
        image=np.array(image_set[i].img)
        row,col=image.shape
        feature_vectors_images = []
        for m in range(0,(row-8),4):
            for n in range(0,(col-8),4):
                patch_array = image[m:m+8,n:n+8].reshape(1,64)[0]
                patch_array=(patch_array - np.mean(patch_array))/np.std(patch_array)
                np.nan_to_num(patch_array)
                #print(patch_array)
                feature_vectors_images.append(patch_array)
        feature_vectors_images=np.array(feature_vectors_images)
        feature_vectors_images=np.nan_to_num(feature_vectors_images)
        kmeans.partial_fit(feature_vectors_images)

    
    return kmeans.cluster_centers_

In [5]:
#vocab = patches_feature(train_img_set)

In [6]:
from numpy.linalg import norm

def get_bags_of_words(image_set):
    vocab = np.load('vocab.npy')
    vocab_length = vocab.shape[0]
    images_histograms = np.zeros((len(image_set), vocab_length))
    feature_vectors_images = []
    for i, image in enumerate(image_set):
        image=np.array(image_set[i].img)
        row,col=image.shape
        feature_vectors_images = []
        for m in range(0,(row-8),4):
            for n in range(0,(col-8),4):
                patch_array = image[m:m+8,n:n+8].reshape(1,64)[0]
                patch_array=(patch_array - np.mean(patch_array))/np.std(patch_array)
                np.nan_to_num(patch_array)
                feature_vectors_images.append(patch_array)
        feature_vectors_images=np.array(feature_vectors_images)
        feature_vectors_images=np.nan_to_num(feature_vectors_images)
        feature_vectors_images=np.array(feature_vectors_images)
        histogram = np.zeros(vocab_length)
        distances = cdist(feature_vectors_images, vocab)  #每一张图片和词袋进行对比
        closest_vocab = np.argsort(distances, axis=1)[:,0]
        indices, counts = np.unique(closest_vocab, return_counts=True)
        histogram[indices] += counts
        histogram = histogram / norm(histogram)
        images_histograms[i] = histogram
    return images_histograms

In [7]:
train_feature = get_bags_of_words(train_img_set)
test_feature = get_bags_of_words(test_img_set)

<ipython-input-6-c19ed22af4bd>:15: RuntimeWarning: invalid value encountered in true_divide
  patch_array=(patch_array - np.mean(patch_array))/np.std(patch_array)


In [8]:
from sklearn.svm import LinearSVC

num_categories = len(categories)
num_train_per_cat = 100

train_labels = [None] * (num_categories * num_train_per_cat) # 1500 * 1 

for i,cat in enumerate(categories):
    for j in range(num_train_per_cat):
        train_labels[i * num_train_per_cat + j] = cat

def svm_classify(train_image_feats, train_labels, test_image_feats):
    clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(train_image_feats, train_labels)
    test_predictions = clf.predict(test_image_feats)
    return test_predictions


In [9]:
test_predictions = svm_classify(train_feature, train_labels, test_feature)

In [10]:
print(test_predictions)

['Street' 'Forest' 'Insidecity' ... 'Office' 'Office' 'industrial']


In [15]:
with open('E:\\run2.txt','a') as f:
    for t,category in enumerate(test_predictions):
        text =str(test_img_set[t].number) +'.jpg'+' '+ category +  '\n'
        f.write(text)